# Тестовое задание для mindset

In [4]:
import pandas as pd
import json

from neo4j import GraphDatabase

In [5]:
try:
    data = pd.read_csv('data_test.csv', sep=';')
except:
    data = pd.read_csv('https://raw.githubusercontent.com/artemvhvn/mindset_test/main/data_test.csv', sep=';')

In [6]:
data.head()

,id события,ФИО участника события 1,ФИО участника события 2
0,189,Галчевская Карина Владимировна,Белоновская Анастасия Семеновна
1,206,Офицеров Олег Романович,Сапожник Борис Валерьевич
2,445,Жандарова Лариса Германовна,Чемодуров Дамир Русланович
3,503,Масимова Яна Дамировна,Мингажетдинов Рамиль Семенович
4,571,Мухтарова Алена Яковлевна,Щербатенко Ольга Робертовна


In [7]:
# переименуем столбцы для удобной работы в jupyter notebook
data.columns = ['event_id', 'name_of_event_participant_1', 'name_of_event_participant_2']

In [8]:
# класс для подключения к neo4j
class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        if self.driver is not None:
            self.driver.close()

# Метод, который передает запрос в БД
    def query(self, query, db=None):
        assert self.driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.driver.session(database=db) if db is not None else self.driver.session()
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response

In [10]:
# подключение
conn = Neo4jConnection(uri="bolt://localhost:7687", user="artemvhvn", password="12345678")
# создаем базу данных
conn.query("CREATE OR REPLACE DATABASE testdb")

[]

In [11]:
# загружаем датасет в бд
query_string = '''
LOAD CSV WITH HEADERS FROM
    'https://raw.githubusercontent.com/artemvhvn/mindset_test/main/data_test.csv'
    AS line FIELDTERMINATOR ';'

MERGE (name:name {name: line['ФИО участника события 1']})
MERGE (id:event_id {event_id: line['id события']})

MERGE (name)-[:IS_IN]->(id)

;

'''
conn.query(query_string, db='testdb')

[]

Создали узлы с ФИО участника и id события. Связь: ФИО - IS_IN -> id. Теперь добавим вторые ФИО участников с такой же связью.

In [12]:
query_string = '''
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/artemvhvn/mindset_test/main/data_test.csv'
AS line FIELDTERMINATOR ';'

MATCH (id:event_id {event_id: line['id события']})

MERGE (name:name {name: line['ФИО участника события 2']})

MERGE (name)-[:IS_IN]->(id)

;

'''
conn.query(query_string, db='testdb')

[]

![graph](graph.png)

In [16]:
query_string = '''
MATCH ()-[r]->()
RETURN count(r)
'''
conn.query(query_string, db='testdb')


[<Record count(r)=10000>]

Всего 10000 связей

In [26]:
query_string = '''
MATCH (n:event_id)
RETURN count(n)
'''
conn.query(query_string, db='testdb')


[<Record count(n)=4985>]

Всего 4985 событий, т.к. в датасете есть дубликаты по id события, вероятно, это события, в которых учавствовало более двух человек одновременно.

In [29]:
data.event_id.nunique()

4985

In [30]:
data[data.event_id.duplicated()].head(3)

,event_id,name_of_event_participant_1,name_of_event_participant_2
367,70049,Федова Анжелика Вадимовна,Вальдовский Альберт Ефимович
487,92995,Кучеренко Ирина Ильинична,Болтик Григорий Максимович
598,117280,Уточкин Евгений Анатольевич,Каганович Лилия Петровна


In [31]:

query_string = '''
MATCH (n)
WHERE n.event_id IN ['70049', '92995', '117280']
RETURN n
'''
conn.query(query_string, db='testdb')


[<Record n=<Node element_id='4:ae6ca6a0-d01b-4755-abf2-ba63d8f52efc:747' labels=frozenset({'event_id'}) properties={'event_id': '70049'}>>,
 <Record n=<Node element_id='4:ae6ca6a0-d01b-4755-abf2-ba63d8f52efc:992' labels=frozenset({'event_id'}) properties={'event_id': '92995'}>>,
 <Record n=<Node element_id='4:ae6ca6a0-d01b-4755-abf2-ba63d8f52efc:1230' labels=frozenset({'event_id'}) properties={'event_id': '117280'}>>]

![bigevents](bigevents.png)

In [32]:
query_string = '''
MATCH (n:name)
RETURN count(n)
'''
conn.query(query_string, db='testdb')


[<Record count(n)=9899>]

Всего получилось 9899 уникальных ФИО участников, т.к. некоторые учавствовали в нескольких событиях.

Посмотрим на 3 участников с наибольшим количеством событий

In [33]:
query_string = '''
MATCH (p)
CALL {
  WITH p
  MATCH (p)--(c)
  RETURN count(c) AS numberOfConnections
}
RETURN p
ORDER BY numberOfConnections DESC
LIMIT 3
'''
conn.query(query_string, db='testdb')

[<Record p=<Node element_id='4:ae6ca6a0-d01b-4755-abf2-ba63d8f52efc:349' labels=frozenset({'name'}) properties={'name': 'Ахромеева Алина Ивановна'}>>,
 <Record p=<Node element_id='4:ae6ca6a0-d01b-4755-abf2-ba63d8f52efc:597' labels=frozenset({'name'}) properties={'name': 'Башнина Антонина Глебовна'}>>,
 <Record p=<Node element_id='4:ae6ca6a0-d01b-4755-abf2-ba63d8f52efc:1638' labels=frozenset({'name'}) properties={'name': 'Медведева Дарья Алексеевна'}>>]

![popnames](popnames.png)

In [34]:
query_string = '''
MATCH (p)--(c)
RETURN p, count(c) AS numberOfConnections
ORDER BY numberOfConnections DESC
LIMIT 5
'''
conn.query(query_string, db='testdb')

[<Record p=<Node element_id='4:ae6ca6a0-d01b-4755-abf2-ba63d8f52efc:349' labels=frozenset({'name'}) properties={'name': 'Ахромеева Алина Ивановна'}> numberOfConnections=50>,
 <Record p=<Node element_id='4:ae6ca6a0-d01b-4755-abf2-ba63d8f52efc:597' labels=frozenset({'name'}) properties={'name': 'Башнина Антонина Глебовна'}> numberOfConnections=14>,
 <Record p=<Node element_id='4:ae6ca6a0-d01b-4755-abf2-ba63d8f52efc:1638' labels=frozenset({'name'}) properties={'name': 'Медведева Дарья Алексеевна'}> numberOfConnections=6>,
 <Record p=<Node element_id='4:ae6ca6a0-d01b-4755-abf2-ba63d8f52efc:1298' labels=frozenset({'name'}) properties={'name': 'Зимнухова Карина Даниловна'}> numberOfConnections=5>,
 <Record p=<Node element_id='4:ae6ca6a0-d01b-4755-abf2-ba63d8f52efc:3582' labels=frozenset({'name'}) properties={'name': 'Диомидов Игорь Ильдарович'}> numberOfConnections=5>]

![compl](cnames.png)

Некоторые участники связаны со многими событиями. Если предположить, что наши события, это какие-то матчи в турнире, то можно сделать вывод о том, как они проходили по турнирной сетке. Человек, с наибольшим количеством событий, возможно, является победителем.

### json

Вводим ФИО участника, получаем файл name_js.json

In [35]:
fname = input()
query_string = f'''
CALL apoc.export.json.query(
    "MATCH (name)-[rel]-(secname) WHERE name.name = $name return name, rel, secname",
    "name_js.json",
    {{params:{{name: '{fname}'}}}}
)
'''
conn.query(query_string, db='testdb')

Пафомова Кира Вадимовна


[<Record file='name_js.json' source='statement: cols(3)' format='json' nodes=6 relationships=3 properties=6 time=99 rows=12 batchSize=-1 batches=0 done=True data=None>]

In [38]:
# считываем файл
data_json = []
path = '/Users/avhvn/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-baa6809c-46bf-4fe1-9285-72e9b66450e5/import/'
try:
    for line in open(path + 'name_js.json', encoding='utf-8-sig'):
        data_json.append(json.loads(line))
except:
    for line in open('name_js.json', encoding='utf-8-sig'):
        data_json.append(json.loads(line))
data_json

[{'name': {'type': 'node',
   'id': '886',
   'labels': ['name'],
   'properties': {'name': 'Пафомова Кира Вадимовна'}},
  'rel': {'type': 'relationship',
   'id': '9797',
   'label': 'IS_IN',
   'start': {'id': '886',
    'labels': ['name'],
    'properties': {'name': 'Пафомова Кира Вадимовна'}},
   'end': {'id': '9577',
    'labels': ['event_id'],
    'properties': {'event_id': '958318'}}},
  'secname': {'type': 'node',
   'id': '9577',
   'labels': ['event_id'],
   'properties': {'event_id': '958318'}}},
 {'name': {'type': 'node',
   'id': '886',
   'labels': ['name'],
   'properties': {'name': 'Пафомова Кира Вадимовна'}},
  'rel': {'type': 'relationship',
   'id': '4169',
   'label': 'IS_IN',
   'start': {'id': '886',
    'labels': ['name'],
    'properties': {'name': 'Пафомова Кира Вадимовна'}},
   'end': {'id': '8323',
    'labels': ['event_id'],
    'properties': {'event_id': '829652'}}},
  'secname': {'type': 'node',
   'id': '8323',
   'labels': ['event_id'],
   'properties': 